<a href="https://colab.research.google.com/github/igors96/Gradient_Boosting_Training_Titanic/blob/main/gb_titanic_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
pd.set_option('display.max_columns', 21)
arquivo = pd.read_csv('/content/train.csv')

In [2]:
arquivo.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
arquivo.dtypes

PassengerId      int64
Survived         int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object

In [4]:
arquivo.drop(['Name', 'Ticket', 'PassengerId'], axis = 1, inplace = True)

In [5]:
faltantes = arquivo.isnull().sum()
faltantes_percentual = (faltantes/len(arquivo['Survived']))*100
print(faltantes_percentual)

Survived     0.000000
Pclass       0.000000
Sex          0.000000
Age         19.865320
SibSp        0.000000
Parch        0.000000
Fare         0.000000
Cabin       77.104377
Embarked     0.224467
dtype: float64


In [6]:
arquivo.drop('Cabin', axis = 1, inplace = True)

In [7]:
import numpy as np
arquivo['Embarked'].replace(np.NaN, 'S', inplace = True)
arquivo['Age'].fillna(arquivo['Age'].median(), inplace = True)

In [8]:
faltantes = arquivo.isnull().sum()
faltantes_percentual = (faltantes/len(arquivo['Survived']))*100
print(faltantes_percentual)

Survived    0.0
Pclass      0.0
Sex         0.0
Age         0.0
SibSp       0.0
Parch       0.0
Fare        0.0
Embarked    0.0
dtype: float64


In [9]:
sex_encode = pd.get_dummies(arquivo['Sex'])
embarked_encode = pd.get_dummies(arquivo['Embarked'])
arquivo.drop(['Sex', 'Embarked'], axis = 1, inplace = True)

In [10]:
concatenado = pd.concat([arquivo, sex_encode, embarked_encode], axis = 1)

In [11]:
concatenado.head()

,Survived,Pclass,Age,SibSp,Parch,Fare,female,male,C,Q,S
0,0,3,22.0,1,0,7.2500,0,1,0,0,1
1,1,1,38.0,1,0,71.2833,1,0,1,0,0
2,1,3,26.0,0,0,7.9250,1,0,0,0,1
3,1,1,35.0,1,0,53.1000,1,0,0,0,1
4,0,3,35.0,0,0,8.0500,0,1,0,0,1


In [12]:
x = concatenado.drop('Survived', axis = 1)
y = concatenado['Survived']

In [13]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score

modelo = GradientBoostingClassifier()
skfold = StratifiedKFold(n_splits = 5)
resultado = cross_val_score(modelo,x,y,cv = skfold, n_jobs = -1)
print(resultado.mean())

0.8226853304877284


In [14]:
import numpy as np
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV

valores_grid = {'learning_rate': [0.1,0.09,0.08,0.07,0.06,0.05,0.04,0.03,0.02,0.01]}

modelo1 = GradientBoostingClassifier(n_estimators = 300)

gridGB = GridSearchCV(estimator = modelo1, param_grid = valores_grid, cv = 5, n_jobs = -1)
gridGB.fit(x,y)

print('Melhor learning rate: ', gridGB.best_estimator_.learning_rate)
print('Acurácia: ', gridGB.best_score_)

Melhor learning rate:  0.09
Acurácia:  0.8350448810495262
